# 🏗️ Реальный мини‑проект AI Engineer + MLOps — Capstone (HARD)

Добро пожаловать в сложный, практико‑ориентированный мини‑проект, который отражает задачи реальных AI‑инженеров и MLOps‑специалистов.

Цель: собрать прототип продакшн‑сервиса ML от данных до деплоя: пайплайн обработки, тренировка с трекингом экспериментов, упаковка модели, REST‑API, пакетный инференс, основы мониторинга и намётки CI/CD.

Тематика проекта: Предсказание оттока клиентов (Churn Prediction) как сервис.

Что вы сделаете:
- Подготовите данные и проверите качество (data validation).
- Построите фичепайплайн и модель (sklearn Pipeline + GridSearch/Opt).
- Зафиксируете параметры/метрики (опционально MLflow).
- Сохраните артефакты модели.
- Сконструируете REST‑API черновик (FastAPI).
- Подготовите пакетный инференс.
- Пропишете Dockerfile (шаблон) и идеи для CI/CD.
- Добавите базовый мониторинг и проверку дрейфа данных.

🔥 Уровень: Продвинутый. Это «ближе к реальной работе», чем учебные задачки.


## 🧭 Архитектура и этапы
```
Данные → Валидация → EDA → Фичи/Пайплайн → Тренировка/Валидация → Трекинг экспериментов
      → Экспорт модели → API (онлайн) + Batch (офлайн) → Контейнеризация → Мониторинг/Дрифт
```

Стек (рекомендации):
- Python 3.9+; NumPy, Pandas, scikit‑learn.
- MLflow (опционально), FastAPI (для API), Uvicorn (локальный сервер).
- Docker (шаблон), pytest (минимальные тесты).

💡 Можно заменить домен (например, мошенничество, отказы оборудования), но сохраните структуру MLOps.


## 🚀 Быстрый старт
1. Подготовьте датасет `data/raw/churn.csv` (или аналогичный табличный): целевая переменная `churn` (0/1).
2. Заполните пути в ячейках ниже (data/raw, data/processed).
3. По желанию включите MLflow (установите `pip install mlflow`).
4. Для API используйте FastAPI (установите `pip install fastapi uvicorn pydantic`).

🏁 Цель — получить работающий прототип и чек‑листы для дальнейшей автоматизации.


In [ ]:
# 🛠️ Импорты и базовые настройки
import os
import json
import numpy as np
import pandas as pd
from pathlib import Path

# Воспроизводимость
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Пути проекта (можно менять под себя)
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
MODELS_DIR = BASE_DIR / 'models'
ARTIFACTS_DIR = BASE_DIR / 'artifacts'
for p in [DATA_DIR, RAW_DIR, PROCESSED_DIR, MODELS_DIR, ARTIFACTS_DIR]:
    p.mkdir(parents=True, exist_ok=True)

print('Структура каталогов готова:', DATA_DIR, MODELS_DIR, ARTIFACTS_DIR)


## 📥 1. Получение и загрузка данных
Подготовьте файл `data/raw/churn.csv` со столбцами: признаки (числовые/категориальные) и целевая `churn` (0/1).

Примеры публичных источников: Kaggle (Telco Customer Churn).

Подсказки:
- Категориальные столбцы храните в `object`/`category`.
- Дата/время — преобразуйте в datetime.


In [ ]:
# 🧩 Ваш код: загрузка данных
csv_path = RAW_DIR / 'churn.csv'
if csv_path.exists():
    df = pd.read_csv(csv_path)
    display(df.head())
    print(df.shape)
else:
    print('⚠️ Файл не найден:', csv_path)
    # Для демонстрации создадим синтетические данные
    n = 2000
    df = pd.DataFrame({
        'age': np.random.randint(18, 80, size=n),
        'tenure_months': np.random.randint(1, 72, size=n),
        'plan_type': np.random.choice(['basic','pro','enterprise'], size=n, p=[0.6,0.3,0.1]),
        'monthly_fee': np.round(np.random.uniform(5, 80, size=n), 2),
        'country': np.random.choice(['UZ','RU','KZ','KG','TJ'], size=n),
        'support_tickets_90d': np.random.poisson(1.4, size=n),
        'churn': np.random.binomial(1, 0.25, size=n)
    })
    display(df.head())
    print('Синтетический набор:', df.shape)

# Сохранить как промежуточный артефакт
PROCESSED_DIR.mkdir(exist_ok=True, parents=True)
df.to_csv(PROCESSED_DIR / 'churn_raw_loaded.csv', index=False)
print('✅ Данные загружены и сохранены в processed.')


## ✅ 2. Базовая валидация данных (Data Validation)
Проверим типы, пропуски, уникальности, базовые инварианты. Это имитация data checks, которые вы вынесете в пайплайн/CI.


In [ ]:
# Проверки
assert 'churn' in df.columns, 'Нет целевой переменной churn'
print('Типы данных:')
display(df.dtypes)
print('Пропуски по столбцам:')
display(df.isna().sum())

# Пример инвариантов
if 'age' in df.columns:
    assert (df['age'] >= 0).all(), 'Возраст содержит отрицательные значения'
if 'monthly_fee' in df.columns:
    assert (df['monthly_fee'] >= 0).all(), 'Тариф содержит отрицательные значения'
print('✅ Базовые проверки пройдены (или провалятся с полезным сообщением).')


## 🔎 3. Быстрый EDA
Сводная статистика, распределения, целевая переменная, перекос классов, корреляции числовых.

Подсказки: используйте `df.describe()`, `value_counts()`, `corr()`.


In [ ]:
display(df.describe(include='all'))
if 'churn' in df.columns:
    print('Распределение целевой:')
    display(df['churn'].value_counts(normalize=True))
print('Числовая корреляция:')
display(df.select_dtypes(include=np.number).corr())
print('🔥 EDA базово выполнен. Продолжите при необходимости.')


## 🧪 4. Фичи и ML‑пайплайн (sklearn Pipeline)
- Разделите признаки на числовые и категориальные.
- Масштабирование числовых, One‑Hot для категориальных.
- Модель: LogisticRegression / RandomForest / XGBoost (при наличии).
- Кросс‑валидация и подбор гиперпараметров.

Подсказка: `ColumnTransformer`, `Pipeline`, `GridSearchCV`.


In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report

target_col = 'churn'
y = df[target_col].astype(int)
X = df.drop(columns=[target_col])

num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()

numeric_pipe = Pipeline(steps=[('scaler', StandardScaler())])
categorical_pipe = Pipeline(steps=[('ohe', OneHotEncoder(handle_unknown='ignore'))])

preprocess = ColumnTransformer(
    transformers=[
        ('num', numeric_pipe, num_cols),
        ('cat', categorical_pipe, cat_cols)
    ]
)

model = LogisticRegression(max_iter=200, random_state=RANDOM_STATE)
clf = Pipeline(steps=[('prep', preprocess), ('model', model)])

param_grid = {
    'model__C': [0.1, 1.0, 5.0],
    'model__penalty': ['l2']
}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
search = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=cv, n_jobs=-1, verbose=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=RANDOM_STATE)
search.fit(X_train, y_train)
print('Лучшие параметры:', search.best_params_)
print('Лучший AUC (CV):', search.best_score_)
best_model = search.best_estimator_
pred_proba = best_model.predict_proba(X_test)[:,1]
auc = roc_auc_score(y_test, pred_proba)
print('AUC (test):', round(auc, 4))
print(classification_report(y_test, (pred_proba>0.5).astype(int)))

# Сохраним метрики
metrics = {'auc_test': float(auc)}
with open(ARTIFACTS_DIR / 'metrics.json', 'w', encoding='utf-8') as f:
    json.dump(metrics, f, ensure_ascii=False, indent=2)
print('✅ Метрики сохранены в artifacts/metrics.json')


## 📒 5. Трекинг экспериментов (опционально MLflow)
Если MLflow установлен, зафиксируйте параметры/метрики/артефакты.

Подсказка: перед запуском можно поднять UI: `mlflow ui` (локально).


In [ ]:
# 🧩 Ваш код: логирование в MLflow (необязательно)
try:
    import mlflow
    import mlflow.sklearn
    mlflow.set_experiment('churn_capstone')
    with mlflow.start_run(run_name='logreg_grid'):
        mlflow.log_params(search.best_params_)
        mlflow.log_metrics({'auc_test': float(auc)})
        # Логируем модель (артефакт)
        mlflow.sklearn.log_model(best_model, artifact_path='model')
    print('✅ Логи отправлены в MLflow')
except Exception as e:
    print('ℹ️ MLflow недоступен или не установлен. Пропускаем. Причина:', e)


## 📦 6. Экспорт модели (артефакты)
Сохраним пайплайн + веса для дальнейшего инференса.


In [ ]:
import joblib
model_path = MODELS_DIR / 'churn_pipeline.joblib'
joblib.dump(best_model, model_path)
print('✅ Модель сохранена в', model_path)
# Сохраним схему фичей (потребуется для API/Batch)
feature_schema = {'num': num_cols, 'cat': cat_cols}
with open(MODELS_DIR / 'feature_schema.json', 'w', encoding='utf-8') as f:
    json.dump(feature_schema, f, ensure_ascii=False, indent=2)
print('✅ Схема фичей сохранена.')


## 🌐 7. REST‑API (FastAPI) — черновик сервиса инференса
Этот код можно вынести в `app.py`. Здесь — минимальный пример.

Команда запуска (локально):
```
uvicorn app:app --host 0.0.0.0 --port 8000
```


In [ ]:
# Пример FastAPI приложения (не запускается автоматически в ноутбуке)
from typing import List, Optional
try:
    from fastapi import FastAPI
    from pydantic import BaseModel
    _FASTAPI_AVAILABLE = True
except Exception:
    _FASTAPI_AVAILABLE = False

class ChurnRequest(BaseModel):
    # Упростим: принимаем произвольный словарь признаков на строку
    records: List[dict]

if _FASTAPI_AVAILABLE:
    app = FastAPI(title='Churn Inference API')
    _pipeline = joblib.load(model_path) if os.path.exists(model_path) else None

    @app.get('/')
    def root():
        return {'status': 'ok'}

    @app.post('/predict')
    def predict(req: ChurnRequest):
        import pandas as pd
        df_in = pd.DataFrame(req.records)
        proba = _pipeline.predict_proba(df_in)[:,1] if _pipeline is not None else []
        return {'proba': [float(x) for x in proba]}
else:
    print('ℹ️ FastAPI не установлен. Пропустите или установите: pip install fastapi uvicorn pydantic')


## 📚 8. Пакетный инференс (Batch)
Скрипт, который читает CSV и пишет предсказания с вероятностями.


In [ ]:
# 🧩 Ваш код: пакетный инференс
def batch_infer(input_csv: Path, output_csv: Path, model_file: Path = model_path):
    pipe = joblib.load(model_file)
    data = pd.read_csv(input_csv)
    probs = pipe.predict_proba(data)[:,1]
    out = data.copy()
    out['churn_proba'] = probs
    out.to_csv(output_csv, index=False)
    return len(out)

# Пример (раскомментируйте, если есть файл):
# batch_infer(PROCESSED_DIR / 'scoring_input.csv', ARTIFACTS_DIR / 'scoring_output.csv')
print('✅ Шаблон пакетного инференса готов.')


## 📦 9. Контейнеризация — Dockerfile (шаблон)
Файл `Dockerfile` (сохраните отдельно в корне проекта):

```
FROM python:3.10-slim
WORKDIR /app
COPY requirements.txt ./
RUN pip install --no-cache-dir -r requirements.txt
COPY . .
EXPOSE 8000
CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8000"]
```

Пример `requirements.txt`:
```
numpy
pandas
scikit-learn
joblib
fastapi
uvicorn
mlflow  # опционально
```


## 🧪 10. Тесты и CI (черновик)
Минимальный `pytest`‑тест для API/модели (сохраните как `tests/test_infer.py`):

```
import joblib, pandas as pd, numpy as np

def test_model_load_and_predict():
    model = joblib.load('models/churn_pipeline.joblib')
    df = pd.DataFrame([{'age':30,'tenure_months':12,'plan_type':'basic','monthly_fee':20,'country':'UZ','support_tickets_90d':0}])
    proba = model.predict_proba(df)[:,1]
    assert np.isfinite(proba).all()
```

CI идеи: запуск тестов и линтеров на PR, сбор Docker‑образа, выгрузка артефактов.


## 🩺 11. Мониторинг и дрейф данных (черновик)
Пример грубого сравнения распределений (KS‑тест) между обучающей и продовой выборками.

Подсказка: для продакшна используйте алерты, дашборды, храните базовую статистику (feature store).


In [ ]:
# 🧩 Ваш код: мониторинг
def ks_drift_stat(ref: pd.Series, cur: pd.Series):
    # Простейшая реализация через эмпирические CDF
    ref_sorted = np.sort(ref.dropna().values)
    cur_sorted = np.sort(cur.dropna().values)

    # унифицированная сетка
    grid = np.unique(np.concatenate([ref_sorted, cur_sorted]))
    def ecdf(x, grid):
        return np.searchsorted(x, grid, side='right') / len(x)
    d = np.max(np.abs(ecdf(ref_sorted, grid) - ecdf(cur_sorted, grid)))
    return float(d)

# Демонстрация на 'monthly_fee' (реф — train, cur — test)
if 'monthly_fee' in X_train.columns:
    d_stat = ks_drift_stat(X_train['monthly_fee'], X_test['monthly_fee'])
    print('KS drift stat (monthly_fee):', round(d_stat, 4))
    if d_stat > 0.2:
        print('⚠️ Возможный дрейф распределения!')
    else:
        print('✅ Дрейф в допустимых пределах (эвристика).')
else:
    print('ℹ️ Поле monthly_fee отсутствует в табличных фичах.')


## 🚢 12. План деплоя (шаблон)
- Сборка Docker образа, пуш в registry.
- Развёртывание в контейнерный оркестратор (Kubernetes/managed).
- Конфиг секрета/переменных окружения (логирование, трекинг).
- Прокси/ingress для API, авто‑масштабирование.
- Мониторинг: лог‑агрегация, метрики (latency, throughput, ошибки), дрейф фичей.
- Канареечный/blue‑green деплой.
- Обратная связь и цикл переобучения.


## 🔥 Сложные задания (для прокачки)
1. Замените LogisticRegression на CatBoost/XGBoost/LightGBM с поиском гиперпараметров.
2. Вынесите фичепайплайн в модуль и добавьте unit‑тесты на трансформеры.
3. Настройте полноценный MLflow (трекер/артефакты), добавьте модельный реестр.
4. Соберите Docker‑образ и поднимите API локально через docker run.
5. Добавьте метрики мониторинга и алерты (Prometheus/Grafana, Sentry).
6. Реализуйте простую проверку данных на входе API (pydantic схемы, строгие типы).
7. Примените модель к живому потоку (имитация через расписание/стрим).
8. Настройте CI (GitHub Actions/GitLab CI): линтеры, тесты, сборка образа, пуш.
9. Реализуйте оценку экономического эффекта (business metrics).
10. Добавьте A/B тест предсказаний на подмножестве трафика.

🏆 После выполнения вы получите проект, похожий на реальные задачи AI Engineer/MLOps.
🔥 Отличная работа!
